# Load the Data Set From s3

In [8]:
import boto3
import pandas as pd
import os
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

In [9]:
# Initialize the S3 client
s3_client = boto3.client('s3')
s3 = boto3.client('s3')
bucket_name = 'keithsbucketaws'
base_key = 'Add_data/'  

In [10]:
files = ['campain_add_data_lingerie.csv']
local_dir = '/tmp'

for file in files:
    s3_key = f"{base_key}{file}" 
    local_path = f"{local_dir}/{file}"
    
    s3.download_file(bucket_name, s3_key, local_path)
    print(f"Downloaded {file} to {local_path}")

Downloaded campain_add_data_lingerie.csv to /tmp/campain_add_data_lingerie.csv


In [57]:
# Load the data into a pandas DataFrame
df = pd.read_csv('/tmp/campain_add_data_lingerie.csv')

EXPLORATORY DATA ANALYSIS
We begin by examining the structure of our data.
We can see that there are NaN values
Some Column Names can be shortened
How many unique Ad Set Names Exist?

In [58]:
# Display the first few rows
df.head()

,Campaign name,Ad set name,Ad name,Amount spent (GBP),Impressions,"CPM (cost per 1,000 impressions)",Link clicks,CPC (cost per link click),CTR (link click-through rate),Purchases,Cost per purchase,Purchases conversion value,Purchase ROAS (return on ad spend),Adds to cart,Reporting starts,Reporting ends
0,NaN,NaN,NaN,141158.75,35366271,3.991338,545420.0,0.258807,1.542204,9089.0,15.530724,350153.08,2.480562,26156.0,2024-01-01,2024-12-31
1,SW | Advantage+ & LAL Sales | UK,SW | Advantage+ & LAL Sales | UK,SW | Advantage+ & LAL Sales | UK,SW | Advantage+ & LAL Sales | UK,773896,4.867204,8653.0,0.435307,1.118109,162.0,23.251296,6861.52,1.821622,847.0,2024-01-01,2024-12-31
2,SW | Advantage+ Sales | Subscription | USA,A+ USA,Video - August Set,2820.37,403869,6.983378,5783.0,0.487700,1.431900,86.0,32.795000,5952.16,2.110418,887.0,2024-01-01,2024-12-31
3,Instagram post: A lingerie club for likeminded...,Instagram Post,Instagram post: A lingerie club for likeminded...,2480.00,762146,3.253970,31669.0,0.078310,4.155241,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-12-31
4,SW | Advantage+ & LAL Sales | UK,Advantage Plus (Exc. Existing Customers),Video - August Set,2321.35,516478,4.494577,4220.0,0.550083,0.817073,105.0,22.108095,4834.56,2.082650,629.0,2024-01-01,2024-12-31


In [59]:
# rename columns for ease.
df = df.rename(columns={'Campaign name': 'campaign_name'})
df = df.rename(columns={'Ad set name': 'ad_set_name'})
df = df.rename(columns={'Amount spent (GBP)': 'amount_spent_gbp'})
df = df.rename(columns={'CPM (cost per 1,000 impressions)': 'cost_per_thousand_impressions'})
df = df.rename(columns={'Link clicks': 'link_clicks'})
df = df.rename(columns={'CPC (cost per link click)': 'cost_per_link_click'})
df = df.rename(columns={'CTR (link click-through rate)': 'link_clickthrough_rate'})
df = df.rename(columns={'Cost per purchase': 'cost_per_purchase'})
df = df.rename(columns={'Purchases conversion value': 'purchases_conversion_value'})
df = df.rename(columns={'Purchase ROAS (return on ad spend)': 'purchase_roas'})
df = df.rename(columns={'Adds to cart': 'adds_to_cart'})
df = df.rename(columns={'Reporting starts': 'reporting_starts'})
df = df.rename(columns={'Reporting ends': 'reporting_ends'})
df.shape

(679, 16)

# Campaign Name
Remove Nan Values. 
Find out how many unique campaign names exist
Is it possible to encode them?

In [60]:
# Remove rows where 'campaign_name' is:
# - The string 'nan' or 'NaN' (case-insensitive)
# - A true NaN (missing value)
df_cleaned = df[
    ~df['campaign_name'].astype(str).str.lower().eq('nan') &  # Check for 'nan' strings
    df['campaign_name'].notna()  # Check for actual NaN values
]

# Optional: Check the result
print(f"Original rows: {len(df)}")
print(f"Rows after cleaning: {len(df_cleaned)}")

# Update the DataFrame
df = df_cleaned


Original rows: 679
Rows after cleaning: 678


# Perform Label Encoding on the different unique campaign names (Come back later and experiment with others like One-Hot)

In [62]:
from sklearn.preprocessing import LabelEncoder

# Initialize the encoder
label_encoder = LabelEncoder()

# Fit and transform the campaign names to labels
df['campaign_name_encoded'] = label_encoder.fit_transform(df['campaign_name'])
print(df[['campaign_name', 'campaign_name_encoded']].head())

                                       campaign_name  campaign_name_encoded
1                   SW | Advantage+ & LAL Sales | UK                     47
2         SW | Advantage+ Sales | Subscription | USA                     52
3  Instagram post: A lingerie club for likeminded...                      5
4                   SW | Advantage+ & LAL Sales | UK                     47
5  Instagram post: A lingerie club for likeminded...                      5


In [63]:
# Display the first few rows
df.head()

,campaign_name,ad_set_name,Ad name,amount_spent_gbp,Impressions,cost_per_thousand_impressions,link_clicks,cost_per_link_click,link_clickthrough_rate,Purchases,cost_per_purchase,purchases_conversion_value,purchase_roas,adds_to_cart,reporting_starts,reporting_ends,campaign_name_encoded
1,SW | Advantage+ & LAL Sales | UK,SW | Advantage+ & LAL Sales | UK,SW | Advantage+ & LAL Sales | UK,SW | Advantage+ & LAL Sales | UK,773896,4.867204,8653.0,0.435307,1.118109,162.0,23.251296,6861.52,1.821622,847.0,2024-01-01,2024-12-31,47
2,SW | Advantage+ Sales | Subscription | USA,A+ USA,Video - August Set,2820.37,403869,6.983378,5783.0,0.487700,1.431900,86.0,32.795000,5952.16,2.110418,887.0,2024-01-01,2024-12-31,52
3,Instagram post: A lingerie club for likeminded...,Instagram Post,Instagram post: A lingerie club for likeminded...,2480.00,762146,3.253970,31669.0,0.078310,4.155241,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-12-31,5
4,SW | Advantage+ & LAL Sales | UK,Advantage Plus (Exc. Existing Customers),Video - August Set,2321.35,516478,4.494577,4220.0,0.550083,0.817073,105.0,22.108095,4834.56,2.082650,629.0,2024-01-01,2024-12-31,47
5,Instagram post: A lingerie club for likeminded...,Instagram Post,Instagram post: A lingerie club for likeminded...,2268.57,570499,3.976466,33897.0,0.066925,5.941641,31.0,73.179677,937.70,0.413344,NaN,2024-01-01,2024-12-31,5
